In [2]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import os
import re

In [4]:
file_path = '../../data/mens/mens_results.csv'

In [17]:
matches = pd.read_csv(file_path)
matches

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score
0,"Dual Match: University of California, Los Ange...",2025-04-26,Emon Van Loben Sels,13.59,Alexander Bernard,13.33,"7-6(5), 5-7, 6-3"
1,"Dual Match: University of California, Los Ange...",2025-04-26,Alexander Hoogmartens,13.00,Jack Anthrop,13.66,"7-5, 0-6, 6-4"
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2"
3,"Dual Match: University of California, Los Ange...",2025-04-26,Preston Stearns,13.08,Aadarsh Tripathi,12.94,"6-3, 2-6, 6-2"
4,"Dual Match: University of California, Los Ange...",2025-04-26,Kaylan Bigun,13.17,William Jansen,13.24,"6-2, 7-5"
...,...,...,...,...,...,...,...
4875,NaN,2013-09-29,Alex Alvarez,0.00,Kaylan Bigun,13.17,"6-2, 6-1"
4876,NaN,2013-09-29,Kaylan Bigun,13.17,Saul Baeza de la Paz,0.00,"6-4, 6-1"
4877,NaN,2013-01-20,Kaylan Bigun,13.17,Mateo Moss,0.00,"6-3, 6-2"
4878,NaN,2013-01-20,Cole Stelse,12.00,Kaylan Bigun,13.17,"6-1, 6-1"


In [18]:
# Set Player Name!
player_name = 'Rudy Quan'
school = "University of California, Los Angeles"
# date = ?

In [19]:
# Helper function to flip scores like "7-6(5), 5-7, 6-3"
def flip_score(score_str):
    if not isinstance(score_str, str):
        return score_str
    flipped = []
    for set_score in score_str.split(', '):
        match = re.match(r'(\d+)-(\d+)(\(\d+\))?', set_score)
        if match:
            p1, p2, tiebreak = match.groups()
            flipped_score = f"{p2}-{p1}"
            if tiebreak:
                flipped_score += tiebreak
            flipped.append(flipped_score)
        else:
            flipped.append(set_score)  # Preserve unrecognized formats
    return ', '.join(flipped)


# UPDATE do not include the comma in final result

In [20]:
def index_csv(data, player_name, school):

    # Filter for Player
    df = data[(data['Player1'] == player_name) | (matches['Player2'] == player_name)].copy()

    
    # Extract opponent name from 'Event Name'
    def get_opponent(event):
        if not isinstance(event, str):
            return None
        teams = re.findall(r'vs\s(.+)', event)
        if school in event and teams:
            return teams[0].strip() if school in event.split('vs')[0] else event.split('vs')[0].replace('Dual Match:', '').strip()
        return None

    # Create opponent_school Column
    df['opponent_school'] = df['Event Name'].apply(get_opponent)

    # Create opponent Column
    df['opponent'] = df.apply(lambda row: row['Player2'] if row['Player1'] in player_name 
                                    else (row['Player1'] if row['Player2'] in player_name 
                                    else None
                                    ), axis=1
                                    )
    
    # UPDATE: Make all names to where the only the first letter of each name is capitalized
    # eg. Aristotelis THANOS --> Aristotelis Thanos
    df['opponent'] = df['opponent'].str.title()

    # Create ucla_result Column
    df['ucla_result'] = df.apply(lambda row: 'Win' if row['Player1'] in player_name 
                                            else ('Loss' if row['Player2'] in player_name 
                                            else None), axis=1
                                            )
    
    # UPDATE: Make Win and Loss --> "W" and "L" AND take into account UNFINISHED matches "UF"
    # UPDATE: Combine result and score column to say result after score
    # eg. 6-3 6-3 W  | 3-6 4-6 L  | 2-6 6-5 UF
    def determine_result(row):
        score = row['Score']
        if pd.isna(score) or 'UF' in str(score).upper():
            return 'UF'

        sets = score.split(',')
        p1_sets_won = 0
        p2_sets_won = 0
        valid_sets = 0

        for set_score in sets:
            set_score = set_score.strip().split('(')[0]
            try:
                p1_games, p2_games = map(int, set_score.strip().split('-'))
            except:
                continue
            valid_sets += 1
            if p1_games > p2_games:
                p1_sets_won += 1
            elif p2_games > p1_games:
                p2_sets_won += 1

        if valid_sets < 2 or p1_sets_won == p2_sets_won:
            return 'UF'

        if row['Player1'] == player_name:
            return 'W' if p1_sets_won > p2_sets_won else 'L'
        elif row['Player2'] == player_name:
            return 'W' if p2_sets_won > p1_sets_won else 'L'
        else:
            return None

    # Result column
    df['ucla_result'] = df.apply(determine_result, axis=1)
    
    # Create column Score (UCLA Perspective)
    df['Score (UCLA Perspective)'] = df.apply(lambda row: flip_score(row['Score']) 
                                                        if row['Player2'] in player_name 
                                                        else row['Score'],
                                                        axis=1
                                                        )
    
    # UPDATE: make date column to mm/dd/yy format
    # eg. 2025-04-26 --> 04/26/25
    # Convert the 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter for matches on or after September 1, 2024
    df = df[df['Date'] >= '2024-09-01']
    df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')


    
    final_df = df[['opponent_school', 
                   'opponent', 
                   'Score (UCLA Perspective)', 
                   'ucla_result',
                   'Date']].rename(columns={'Score (UCLA Perspective)': 'score',
                                            'Date': 'date'
                                            })

    return final_df

In [21]:
index_csv(matches, player_name, school)

# UPDATE: Output to csv in index_page folder

,opponent_school,opponent,score,ucla_result,date
2,The Ohio State University,Aidan Kim,"6-7(3), 2-6",L,04/26/2025
10,Michigan State University,Aristotelis Thanos,"4-6, 3-1",UF,04/25/2025
17,University of Michigan,William Cooksey,"6-3, 5-6",UF,04/24/2025
23,University of Nebraska-Lincoln,Calvin Mueller,"3-6, 6-3, 6-1",W,04/19/2025
25,University of Wisconsin-Madison,Michael Minasyan,"6-2, 3-6",UF,04/17/2025
32,Michigan State University,Ozan Baris,"6-1, 6-2",W,04/12/2025
36,University of Michigan,William Cooksey,"6-3, 6-3",W,04/10/2025
44,Pennsylvania State University,Charl Morgan,"6-3, 6-3",W,04/05/2025
48,Ohio State University,Alexander Bernard,"6-7(5), 6-0",UF,04/03/2025
52,The Ohio State University,Alexander Bernard,"6-7(5), 6-0, 1-3",L,04/03/2025


In [29]:
edited = index_csv(matches, player_name, school)

unfinished = edited[edited["ucla_result"] == "UF"]

len(unfinished)

4